<div style="text-align: center;">
    <h1>Assignment 02</h1>
    <h1>孙健玮      12440018</h1>
</div>

# 1. 任务描述
1. 使⽤PyTorch下载MNIST数据集，包括训练集和测试集。
2. 使⽤PyTorch分别实现Logistic Regression模型和MLP（多层感知机）模型。
3. 分别使⽤Logistic Regression模型和MLP模型进⾏训练，调试参数并获得较好的训练结果。
4. 分别记录训练集的Loss并画出曲线。
5. 针对测试集使⽤Accuracy评估模型的性能，讨论两个模型的训练结果并给出⾃⼰的结论。

# 2. 编程模块

## 2.1 数据加载

In [8]:
import torch
from torchvision import transforms, datasets

# 定义数据预处理操作
transform = transforms.Compose([
    transforms.ToTensor(),  # 将图像转换为Tensor
    transforms.Normalize((0.5,), (0.5,))  # 对图像进行标准化处理
])

# 下载训练集MNIST手写数字数据集
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# 加载数据
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

## 2.2 定义模型

In [ ]:
import torch.nn as nn

# Logistic Regression模型
class LogisticRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self). __init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.linear = nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        return self.linear(x)
    
# MLP模型
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

## 2.3 训练模型

In [ ]:
import torch.optim as optim

def train_model(model, train_loader, criterion, optimiser, num_epochs):
    """训练模型
    :param model: 模型
    :param train loader
    :训练数据加载器
    :param criterion
    :损失函数:param optimizer
    :优化器:param num_epochs
    :训练轮数:return
    :训练过程中的损失列表
    """
    model.train()   # 将模型设置为训练模式
    losses = []    # 保存每个epoch的loss
        
    for epoch in range(num_epochs):
        epoch_loss = 0.0
        for i, (images, labels) in enumerate(train_loader):
            # 将图片展平为向量，适用于全连接网络
            images = images.view(-1, 28*28)
            
            # 前向传播
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # 反向传播和优化
            optimiser.zero_grad()
            loss.backword()
            optimiser.step()
            
            # 记录损失
            epoch_loss += loss.item()
            if (i + 1) % 100 == 0:
                print(f'Eposh [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}, Loss: {loss.item():.4f}')
                
        # 记录每个epoch的平均损失
        losses.append(epoch_loss / len(train_loader))
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss {epoch_loss / len(train_loader):.4f}')
        
    return losses
            

## 2.4 测试模块

In [ ]:
def test_model(model, test_loader):
    """测试模型
    :param model: 模型
    :param test_loader: 测试数据加载器
    :return: 测试准确率
    """
    model.eval() # 将模型设置为测试模式
    correct = 0
    total = 0
    
    with torch.no_grad():   # 测试过程中不需要计算梯度
        for images, labels in test_loader:
            images = images.view(-1, 28*28)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    accuracy = 100 * correct / total
    print(f'Accuracy of the model on the test images: {accuracy}')
    return accuracy

## 2.5 主程序

In [ ]:
if __name__ == "__main__":
    # 定义模型参数
    input_dim = 28 * 28
    output_dim = 10
    hidden_dim = 28 * 28
    
    # 学习率
    lr = 0.01
    
    # 定义模型
    logistic_model = LogisticRegression(input_dim, output_dim)
    mlp_model = MLP(input_dim, hidden_dim, output_dim)
    
    # 定义损失函数和优化器
    criterion = nn.CrossEntropyLoss()   # 使用交叉熵损失函数
    logistic_SGD = optim.SGD(logistic_model.parameters, lr)
    mlp_SGD = optim.SGD(mlp_model.parameters, lr)